## Data Processing


In [1]:
from predicate_detection.dataProcessing import read_file, model_builder
from importlib import reload 
import numpy as np
import tensorflow as tf
from time import time

In [2]:
file = "./data/CoNLL2009-ST-English-train.txt"
reader = read_file(file)


pred_to_fit, pred_not = reader.find_all_predicates(occ_greater_than=50)  # I fit on my model just the predicate seen more than 50 times
print("total numeber of fittable predicates are: ", len(pred_to_fit))
pos, pos_pos = reader.find_all_POS() # ritorna la lista di possibili POS e il contatore di quanti predicati si trovano in quel POS
print("possible POS have length: ", len(pos_pos))
max_len_in_data = reader.max_length_sentence()
print("maximum length of sentence is ", max_len_in_data)
len_sent = max_len_in_data

sents = reader.read_sentences(100) ## just read 10 sentences from the train
## vediamo un po' il dataset
i = 1
reader.print_sentence(sents[i])
print(*sents[i], sep="\n")

### il modello serve a creare i dati input per il DNN
model = model_builder(pos_pos, max_length = len_sent)
list_of_words = model.load_model("./data/glove.6B.100d.txt")
#model("the")

X_emb, sent_len, pred, X_POS = model.creation(sents, max_length= len_sent)

total numeber of fittable predicates are:  733
possible POS have length:  48
maximum length of sentence is  144
Ms. Haag plays Elianti .
['1', 'Ms.', 'ms.', 'ms.', 'NNP', 'NNP', '_', '_', '2', '2', 'TITLE', 'TITLE', '_', '_', '_\n']
['2', 'Haag', 'haag', 'haag', 'NNP', 'NNP', '_', '_', '3', '3', 'SBJ', 'SBJ', '_', '_', 'A0\n']
['3', 'plays', 'play', 'play', 'VBZ', 'VBZ', '_', '_', '0', '0', 'ROOT', 'ROOT', 'Y', 'play.02', '_\n']
['4', 'Elianti', 'elianti', 'elianti', 'NNP', 'NNP', '_', '_', '3', '3', 'OBJ', 'OBJ', '_', '_', 'A1\n']
['5', '.', '.', '.', '.', '.', '_', '_', '3', '3', 'P', 'P', '_', '_', '_\n']


install gensim to make word embeddings

In [3]:
sents = reader.read_sentences(1000)
X, sent_len, pred, X_POS = model.creation(sents, max_length= len_sent)

In [4]:

## model parameter
batch_size = 1000
LAYERS = 1
hidden_rapr_Bi = 30

# Data summary
max_length_sentences = len_sent
dim_feature = 2

# place holders dimentions
dimension_embeddings = 100
dimension_POS = 48
POS_emb_dim = 16



tf.reset_default_graph()
g1 = tf.Graph()
with g1.as_default():
    ## Inputs
    one_hot_POS = tf.placeholder(tf.float32, shape = (batch_size, max_length_sentences, dimension_POS), name= "POS")
    embeddings = tf.placeholder(tf.float32, shape=(batch_size, max_length_sentences , dimension_embeddings), name="embeddings")
    sequence_lengths = tf.placeholder(tf.int32, shape = [batch_size])
    
    ## randomized embeddings for POS
    W_emb_pos = tf.get_variable("W_emb_pos", shape = [dimension_POS , POS_emb_dim], dtype = tf.float32)
    POS = tf.reshape(one_hot_POS, shape = [-1, dimension_POS ])
    POS_emb_flat = POS @ W_emb_pos    
    POS_emb = tf.reshape(POS_emb_flat, shape = [ batch_size , max_length_sentences, POS_emb_dim])
    
    # inputs to the Bi-lstm
    inputs = tf.concat([embeddings, POS_emb], axis = -1)

    """
    ## Bi-Lstm for cudnn
    bi_lstm = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers = LAYERS, num_units = hidden_rapr_Bi,
                                             direction = 'bidirectional',
                                             dropout = 0.2,
                                             seed = 123, dtype = tf.float32, name="Bi-Lstm")

    outputs_Bi_Lstm, output_states = bi_lstm(inputs)

    context_flat = tf.reshape(outputs_Bi_Lstm, shape = [-1,2*hidden_rapr_Bi ])
    """

    ## Bi-lstm two layers for cpu 
    fw_cell_list = [tf.nn.rnn_cell.LSTMCell(hidden_rapr_Bi*i) for i in range(2,0,-1)]
    bw_cell_list = [tf.nn.rnn_cell.LSTMCell(hidden_rapr_Bi*i) for i in range(2,0,-1)]
    fw_cells = tf.nn.rnn_cell.MultiRNNCell(fw_cell_list)
    bw_cells = tf.nn.rnn_cell.MultiRNNCell(bw_cell_list)
    outputs , outputs_states = tf.nn.bidirectional_dynamic_rnn(fw_cells, bw_cells, 
                                                               inputs,
                                                               sequence_length=sequence_lengths, dtype= tf.float32)
    
    output = tf.concat(outputs, 2)
    context_flat = tf.reshape(output, shape = [-1,2*hidden_rapr_Bi ])
    
    ## Last layer weigths
    W_out = tf.get_variable("W_out", shape = [2*hidden_rapr_Bi , dim_feature], dtype = tf.float32)
    b_out = tf.get_variable("b_out", shape = [dim_feature], dtype = tf.float32, initializer=tf.zeros_initializer())


    pred_flat = tf.matmul(context_flat, W_out) + b_out
    pred = tf.reshape(pred_flat, shape= [batch_size, max_length_sentences, dim_feature])


    ###### Losses and Optimizer
    labels = tf.placeholder(tf.int32, shape=[batch_size, max_length_sentences, dim_feature], name= "labels")

    losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels=labels)

    #mask
    
    mask = tf.sequence_mask(sequence_lengths, max_length_sentences)
    losses = tf.boolean_mask(losses, mask)
    loss = tf.reduce_mean(losses)

    ### Optimizer
    lr = 0.01
    optimizer = tf.train.AdamOptimizer(lr)
    train_op = optimizer.minimize(loss)

    ### labels pred
    labels_pred = tf.cast(tf.argmax(pred, axis=-1), dtype= tf.int32, name="pred_labels")


Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [ ]:
start = time()
sents = reader.read_sentences(1000)
X_emb, sent_len, pred, X_POS = model.creation(sents, max_length= len_sent)

feed_dict={one_hot_POS: X_POS[:batch_size],
           embeddings: X_emb[:batch_size],
           sequence_lengths:sent_len[:batch_size],
           labels: pred[:batch_size]
          }

with g1.as_default():
    for ite in range(10000):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as s:
            s.run(tf.initializers.global_variables())
            l , _ = s.run([loss, train_op], feed_dict=feed_dict)
            if ite % 100==1:
                print("loss function: ", l)
                print("execution time: ", time()-start)
            
print("execution time: ", time()-start)

loss function:  0.66994506
execution time:  14.034213304519653
loss function:  0.707205
execution time:  562.0122532844543
